In [ ]:
import random
from animalai.environment import AnimalAIEnvironment
from wrappers import CustomUnityToGymWrapper
from animalai_agent import AnimalAIVectorhashAgent
from vectorhash import build_vectorhash_architecture
from smoothing import PolynomialSmoothing
import random
from shifts import RatShiftWithCompetitiveAttractorDynamics

### vhash
shapes = [(5, 5, 5), (8, 8, 8)]
model = build_vectorhash_architecture(
    shapes,
    N_h=1200,
    input_size=84 * 84,
    initalization_method="by_sparsity",
    smoothing=PolynomialSmoothing(k=1.5),
    shift=RatShiftWithCompetitiveAttractorDynamics(
        sigma_xy=0.3, sigma_theta=0.3, inhibition_constant=0.004, delta_gamma=1
    ),
    limits=(40, 40, 360),
    relu=True,
    percent_nonzero_relu=0.2,
)


### animalai
aai_seed = 0
port = 5005 + random.randint(
    0, 1000
)  # uses a random port to avoid problems if a previous version exits slowly
# env_path = "/Users/Ile-Maurice/Desktop/MacOS/MacOS"
env_path = "/home/ezrahuang/AAI/LINUX/AAI.x86_64"
configuration_file = "./animal_ai_environments/yroom.yaml"
watch = True

aai_env = AnimalAIEnvironment(
    file_name=env_path,  # Path to the environment
    seed=aai_seed,  # seed for the pseudo random generators
    arenas_configurations=configuration_file,
    play=False,  # note that this is set to False for training
    base_port=port,  # the port to use for communication between python and the Unity environment
    inference=False,  # set to True if you want to watch the agent play
    useCamera=True,  # set to False if you don't want to use the camera (no visual observations)
    resolution=84,
    useRayCasts=False,  # set to True if you want to use raycasts
    no_graphics=False,  # set to True if you don't want to use the graphics ('headless' mode)
    timescale=1,
)

env = CustomUnityToGymWrapper(
    aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True
)  # the wrapper for the environment
agent = AnimalAIVectorhashAgent(model, env)

In [ ]:
"""
0 - nothing

1 - rotate right by 6 degrees

2 - rotate left by 6 degrees

3 - accelerate forward

4 - accelerate forward and rotate CW by 6 degrees

5 - accelerate forward and rotate CCW by 6 degrees

6 - accelerate backward

7 - accelerate backward and rotate CW by 6 degrees

8 - accelerate backward and rotate CCW by 6 degrees
"""

forward_10 = [3] * 10
backward_10 = [6] * 10
forward_3 = [3] * 3
backward_3 = [6] * 3
turn_90_cw = [1] * 15  # = 90 / 6
turn_90_ccw = [2] * 15  # = 90 / 6

visible_15 = [True] * 15
visible_10 = [True] * 10
visible_3 = [True] * 3
not_visible_15 = [False] * 15
not_visible_10 = [False] * 10
not_visible_3 = [False] * 3

path = (
    forward_10
    + turn_90_cw
    + forward_3
    + turn_90_cw
    + forward_10
    + turn_90_cw
    + forward_3
    + turn_90_cw
    + forward_10
)

visibles = (
    visible_10
    + visible_15
    + not_visible_3
    + not_visible_15
    + not_visible_10
    + not_visible_15
    + not_visible_3
    + not_visible_15
    + visible_10
)

noise_list = [[]] * len(path)
assert (
    len(path) == len(visibles) == len(visibles)
), "Path, visibles, and noise must be the same length"
print(f"total length: {len(path)}, path={list(zip(path, visibles, noise_list))}")

In [ ]:
from animalai.environment import AnimalAIEnvironment
from wrappers import CustomUnityToGymWrapper
from animalai_agent import AnimalAIVectorhashAgent
from vectorhash import build_vectorhash_architecture
from smoothing import PolynomialSmoothing
from shifts import RatShiftWithCompetitiveAttractorDynamics
from graph_utils import plot_errors_on_axes
import os
import pickle
import itertools

if not os.path.exists("kidnapping_tests"):
    os.makedirs("kidnapping_tests")


def build_env(port):
    # env_path = "/Users/Ile-Maurice/Desktop/MacOS/MacOS"
    env_path = "/home/ezrahuang/AAI/LINUX/AAI.x86_64"
    configuration_file = "./animal_ai_environments/yroom.yaml"

    aai_env = AnimalAIEnvironment(
        file_name=env_path,  # Path to the environment
        seed=aai_seed,  # seed for the pseudo random generators
        arenas_configurations=configuration_file,
        play=False,  # note that this is set to False for training
        base_port=port,  # the port to use for communication between python and the Unity environment
        inference=False,  # set to True if you want to watch the agent play
        useCamera=True,  # set to False if you don't want to use the camera (no visual observations)
        resolution=84,
        useRayCasts=False,  # set to True if you want to use raycasts
        no_graphics=False,  # set to True if you don't want to use the graphics ('headless' mode)
        timescale=1,
    )

    env = CustomUnityToGymWrapper(
        aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True
    )  # the wrapper for the environment
    return env


aai_seed = 0
port_base = 5000
device = "cuda"

store_methods = ["Always", "When New"]
shift_methods = ["Additive", "Multiplicative"]

shapes = [(5, 5, 5), (8, 8, 8)]
N_h = 600
smoothing = PolynomialSmoothing(k=1.5)
shift = RatShiftWithCompetitiveAttractorDynamics(
    sigma_xy=0.3,
    sigma_theta=0.3,
    inhibition_constant=0.004,
    delta_gamma=1,
    device=device,
)
limits = (40, 40, 360)

model = build_vectorhash_architecture(
    shapes,
    N_h=N_h,
    initalization_method="by_sparsity",
    smoothing=smoothing,
    shift=shift,
    limits=limits,
    relu=True,
    percent_nonzero_relu=0.2,
    device=device,
    input_size=84*84
)

In [ ]:
from animalai_agent import kidnapping_test
test_methods = itertools.product(store_methods, shift_methods)

for o, [store_method, shift_method] in enumerate(test_methods):
    # each i is an array with [alg_method, additive/multiplicative]
    assert store_method in ["Always", "When New"]
    assert shift_method in ["Additive", "Multiplicative"]

    env = build_env(port_base + o)
    store_new = True if store_method == "When New" else False
    additive_shift = True if shift_method == "Additive" else False
    model.reset()
    agent = AnimalAIVectorhashAgent(
        vectorhash=model, env=env, store_new=store_new, additive_shift=additive_shift
    )
    history = kidnapping_test(agent, path, noise_list, visibles)
    agent_history = agent.history

    with open(f"kidnapping_tests/kidnapped_history_{o}.pkl", "wb") as f:
        pickle.dump(history, f)

    env.close()

In [ ]:
from animalai_agent_history import (
    VectorhashAgentHistory,
    VectorhashAgentKidnappedHistory,
)
from matplotlib import pyplot as plt

for o, [store_method, shift_method] in enumerate(itertools.product(store_methods, shift_methods)):
    with open(f"kidnapping_tests/kidnapped_history_{o}.pkl", "rb") as f:
        history: VectorhashAgentKidnappedHistory = pickle.load(f)

    fig, ax = plt.subplots(1, 1, figsize=(10, 5))

    ax.set_title(
        f"Kidnapping test: store_method={store_method}; shift_method={shift_method}"
    )
    plot_errors_on_axes(history, ax, visible=visibles)
    ax.legend()
    fig.savefig(
        f"kidnapping_tests/kidnapped_history_{o}.png",
        dpi=150,
    )


In [ ]:
for o, [store_method, shift_method] in enumerate(itertools.product(store_methods, shift_methods)):
    with open(f"kidnapping_tests/kidnapped_history_{o}.pkl", "rb") as f:
        agent_history: VectorhashAgentHistory = pickle.load(f)
    ani = agent_history.make_image_video()
    ani.save(
        f"kidnapping_tests/test_{o}.gif",
        progress_callback=lambda current, total: print(f"frame {current+1}/{total}"),
        dpi=150,
    )